In [ ]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_ibm_runtime import Session, QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from matplotlib import pyplot as plt
from datetime import datetime, timezone
import numpy as np
from numpy import linalg as la
import os, csv

# API tokens for IBM Quantum Platform. Updated on 04-Dec-2024.

## Account Information

### KQC_Pharmcadd
### ibm-q-kqc / pharcadd / research OR ibm-q / open / main
token = 'e2b36571a4a8ed3720a30c8d7b2d59b55347beebe48614832e74a156a3669e6179d306b2ff3727f08ef95b99c1166add2a774e80fff85405abe38a0da1eb1c8c'

instances = [marrakesh, fez (156), torino (133), brussels, nazca, strasbourg, {kyiv, brisbane, sherbrooke} (127)]

### my account: ichi@kaist
### ibm-q-skku / kaist / kaist-graduate OR ibm-q / open / main
token = '23b4b9e4f3507d73dd25691b5b96bc6a70ab7798ea6821b91db525151323338264a62e5adfff61edb604f62fcaa51a4484a54450bb239b8c52e8d1ccd76344c2'

instances = [marrakesh, fez (156), torino (133), brussels, nazca, strasbourg, {kyiv, brisbane, sherbrooke} (127)]

Usage limits (open): Monthly usage is limited up to 10 minutes, refreshes on the first of each month. At most 3 pending workloads at a time.

## Load QiskitRuntimeService

In [ ]:
# Show the list of the saved accounts
saved_accounts = QiskitRuntimeService.saved_accounts()

#for key, value in saved_accounts.items():
#    print(key, value)

In [ ]:
# Load account and generate QiskitRuntimeService
default_account = saved_accounts['kaist_ichi9505']
service = QiskitRuntimeService(channel=default_account['channel'],token=default_account['token'])

# Functions

In [ ]:
# <name>: <type> = <default> -> <return_type>
def generate_QST_circuit(qc: QuantumCircuit, meas_basis: list[str]) -> QuantumCircuit:
    qc_qst = qc.copy_empty_like(name='meas_basis')
    
    gates = {'x':qc_qst.x,
            'h':qc_qst.h,
            's':qc_qst.s,
            'sdg':qc_qst.sdg}
    basis_conv = {'x+':['h'],
                  'x-':['h','x'],
                  'y+':['sdg','h'],
                  'y-':['sdg','h','x'],
                  'z+':[None],
                  'z-':['x']}
    
    for qubit, mbasis in enumerate(meas_basis):
        for cg in basis_conv[mbasis]:
            if cg is None:
                pass
            else:
                gates[cg](qubit)
    
    return qc_qst

def generate_QDT_circuit(qc: QuantumCircuit, prep_state: list[str]) -> QuantumCircuit:
    qc_qdt = qc.copy_empty_like(name='state_prep')
    
    gates = {'x':qc_qdt.x,
            'h':qc_qdt.h,
            's':qc_qdt.s,
            'sdg':qc_qdt.sdg}
    basis_conv = {'x+':['h'],
                  'x-':['x','h'],
                  'y+':['h','s'],
                  'y-':['x','h','s'],
                  'z+':[None],
                  'z-':['x']}
    
    for qubit, pstate in enumerate(prep_state):
        for cg in basis_conv[pstate]:
            if cg is None:
                pass
            else:
                gates[cg](qubit)
    
    return qc_qdt

def generate_basis_list(blist:list[list[str]],basis:list[str]=['x+','x-','y+','y-','z+','z-']) -> list[list[str]]:
    return [bpre+[badd] for bpre in blist for badd in basis]

## conduct quantum experiments on ibm backend : QST

In [ ]:
#backend = FakeKyiv()
backend = service.least_busy() #get_backend('ibm_strasbourg') #least_busy()
print(f'The chosen backend: {backend.name}')

basis_gates = [instruction for instruction in backend.configuration().supported_instructions if len(instruction) < 5]
print(f'The basis_gates of {backend.name}: {basis_gates}')

target_qubits = [6,7]

d = 2**len(target_qubits)
shots_per_experiment = 8000

pm = generate_preset_pass_manager(optimization_level=0, backend=backend, initial_layout=target_qubits)

In [ ]:
meas_basis_1 = ['x+','x-','y+','y-','z+','z-']
meas_basis_list = []
for qubit in range(len(target_qubits)):
    if qubit==0:
        meas_basis_list.extend([[m] for m in meas_basis_1])
    else:
        meas_basis_list = generate_basis_list(meas_basis_list, meas_basis_1)

print(len(target_qubits))
isa_circuits_qst = []    
for meas_basis in meas_basis_list:
    print(meas_basis)
    qr = QuantumRegister(len(target_qubits))
    qc = QuantumCircuit(qr)

    qc.h(0)
    qc.cx(0,1)
    qc.barrier()
    qc.compose(generate_QST_circuit(qc, meas_basis), inplace=True)
    qc.measure_all()
    
    isa_circuits_qst.append(pm.run(qc))
    qc.draw('mpl', filename=f"../data/qst_{''.join(meas_basis)}.png", initial_state=True)

print(len(isa_circuits_qst))
print('QST circuit generation is done.')

In [ ]:
meas_all_zeros = '0'*len(target_qubits)
qst_measurement_counts = []
with Session(backend=backend) as session:
    # circuit, measure
    sampler = Sampler(session=session)
    for qc_qst in isa_circuits_qst:
        job = sampler.run([qc_qst], shots=shots_per_experiment)

        pub_results = job.result()[0]['__value__']['data'] #.data
        pub_counts = pub_results.meas.get_counts() #MEAS.get_counts()

        if meas_all_zeros not in pub_counts.keys():
            pub_counts[meas_all_zeros] = 0

#    for outcome in range(d):
#        base2outcome = np.binary_repr(outcome,width=len(target_qubits))
#        if base2outcome not in pub_counts.keys():
#            pub_counts[base2outcome] = 0
        
        qst_measurement_counts.append(pub_counts[meas_all_zeros])
        
    qst_measurement_probs = qst_measurement_counts / np.sum(qst_measurement_counts)

print("{0:<8}{1:^3}{2}".format('basis','|','Probabilities'))
for i, mb in enumerate(meas_basis_list):
    print('{0:>8}{1:^3}{2}'.format(','.join(mb),'|',qst_measurement_probs[i]))

print('{0:>8}{1:^3}{2}'.format('SUM','|',np.sum(qst_measurement_probs)))
print(measurement_probs)

## conduct quantum experiments on ibm backend : QDT

In [ ]:
#backend = FakeKyiv()
backend = service.least_busy() #get_backend('ibm_strasbourg') #least_busy()
print(f'The chosen backend: {backend.name}')

basis_gates = [instruction for instruction in backend.configuration().supported_instructions if len(instruction) < 5]
print(f'The basis_gates of {backend.name}: {basis_gates}')

target_qubits = [6,7]

d = 2**len(target_qubits)
shots_per_experiment = 8000

pm = generate_preset_pass_manager(optimization_level=0, backend=backend, initial_layout=target_qubits)

In [ ]:
prep_state_1 = ['x+','x-','y+','y-','z+','z-']
prep_state_list = []
for qubit in range(len(target_qubits)):
    if qubit==0:
        prep_state_list.extend([[s] for s in prep_state_1])
    else:
        prep_state_list = generate_basis_list(prep_state_list, prep_state_1)

isa_circuits_qdt = []
for prep_state in prep_state_list:
    qr = QuantumRegister(len(target_qubits))
    qc = QuantumCircuit(qr)

    qc.compose(generate_QDT_circuit(qc, prep_state), inplace=True)
    qc.barrier()
    qc.measure_all()
    
    isa_circuits_qdt.append(pm.run(qc))
#    qc.draw('mpl', filename=f"../data/qdt_{''.join(prep_state)}.png", initial_state=True)

print(len(isa_circuits_qdt))
print('QDT circuit generation is done.')

In [ ]:
outcomes_bitstr = []
for outcome in range(d):
    outcomes_bitstr.append(np.binary_repr(outcome,width=len(target_qubits)))
    
qdt_measurement_counts = []
with Session(backend=backend) as session:
    # circuit, measure
    sampler = Sampler(session=session)
    for qc_qdt in isa_circuits_qdt:
        job = sampler.run([qc_qdt], shots=shots_per_experiment)

        pub_results = job.result()[0]['__value__']['data'] #.data
        pub_counts = pub_results.meas.get_counts() #MEAS.get_counts()

        for outcome in outcomes_bitstr:
            if outcome not in pub_counts.keys():
                pub_counts[outcome[-1::-1]] = 0 # little-endian -> big-endian
        
        qdt_measurement_counts.append(pub_counts)

# big endian
qdt_measurement_probs = []
for i, counts_basis in enumerate(qdt_measurement_counts):
    qdt_measurement_probs.append(counts_basis)
    for outcome in outcomes_bitstr:
        qdt_measurement_probs[i][outcome] /= shots_per_experiment
        
space = ' '*(12-len(target_qubits))
print("{0:<8}{1:^3}{2:<5}{3}".format('Input','|',' ','Probabilities'))
print("{0:<8}{1:^3}{2:<5}{3}".format('State','|','sum',space.join(outcomes_bitstr)))
print(f"{'-'*80}", end='')

for i, sp in enumerate(prep_state_list):
    print('\n{0:>8}{1:^3}{2:<5.1f}'.format(','.join(sp),'|',np.sum([prob for prob in qdt_measurement_probs[i].values()])),end='')
    for prob in qdt_measurement_probs[i].values():
        print('{0:<11.6f}'.format(prob), end=' ')

### Backend Properties

In [ ]:
backend_properties = backend.properties().to_dict()

print('==============================')
print("|      Calibration Data      |")
print('------------------------------')

print("Backend name:", backend_properties['backend_name'])
print("Last calibrated date:", backend_properties['last_update_date'])

for i, qq in enumerate(target_qubits):
    rdout_err = backend_properties['qubits'][qq][4]['value'] * 100
    id_err = backend_properties['gates'][qq]['parameters'][0]['value'] * 100
    rz_err = backend_properties['gates'][qq+127]['parameters'][0]['value'] * 100
    sx_err = backend_properties['gates'][qq+127*2]['parameters'][0]['value'] * 100
    x_err = backend_properties['gates'][qq+127*3]['parameters'][0]['value'] * 100
    
    print("========================================================================================")
    print("qubit {0} | readout_error (%): {1}".format(qq,rdout_err))
    print("qubit {0} | id_error (%): {1}".format(backend_properties['gates'][qq]['qubits'][0], id_err))
    print("qubit {0} | rz_error (%): {1}".format(backend_properties['gates'][qq+127]['qubits'][0], rz_err))
    print("qubit {0} | sx_error (%): {1}".format(backend_properties['gates'][qq+127*2]['qubits'][0], sx_err))
    print("qubit {0} | x_error (%): {1}".format(backend_properties['gates'][qq+127*3]['qubits'][0],x_err))

    if not i:
        isdirect = False
        target = target_qubits[i+1]
        for ecr_spec in backend_properties['gates'][127*4:]:
            if (qq in ecr_spec['qubits']) and (target in ecr_spec['qubits']):
                isdirect = True
                print("qubits {0} | ecr(cx?)_error (%): {1}".format([qq, target],ecr_spec['parameters'][0]['value']))
                
        if not isdirect:
            print("qubits [{0}, {1}] are not directly connected.".format(qq,target))
                
print('------------------------------')
print("|            Done            |")
print('==============================')

## function trash bin

In [ ]:
def generate_inverse_unitary_gate(qc:QuantumCircuit, sequence: list[int]) -> QuantumCircuit:
    qc_unitary_inverse = qc.copy_empty_like(name='C_u_inv')
    
    unitaries = {0:np.eye(2),
                1:np.array([[0,1],[1,0]]),
                2:np.array([[0,-1j],[1j,0]]),
                3:np.array([[1,0],[0,-1]]),
                4:np.array([[1,1],[1,-1]] / np.sqrt(2)),
                5:np.array([[1,0],[0,1j]]),
                6:np.array([[1,-1],[1,1]] / np.sqrt(2)),
                7:np.array([[-1j,1j],[1j,1j]] / np.sqrt(2)),
                8:np.array([[1,1],[-1,1]] / np.sqrt(2)),
                9:np.array([[0,1j],[1,0]]),
                10:np.array([[0,1],[1j,0]]),
                11:np.array([[1,0],[0,-1j]]),
                12:np.array([[1,0],[0,-1j]]),
                13:np.array([[0,-1j],[1,0]]),
                14:np.array([[0,-1],[1j,0]]),
                15:np.array([[1,0],[0,1j]]),
                16:np.array([[1,1j],[1,-1j]] / np.sqrt(2)),
                17:np.array([[1,1],[1j,-1j]] / np.sqrt(2)),
                18:np.array([[1,-1j],[1,1j]] / np.sqrt(2)),
                19:np.array([[1,1],[-1j,1j]] / np.sqrt(2)),
                20:np.array([[1,-1],[1,1]] / np.sqrt(2)),
                21:np.array([[1,-1j],[1,1j]] / np.sqrt(2)),
                22:np.array([[-1j,1],[1j,1]] / np.sqrt(2)),
                23:np.array([[1,1j],[-1,1j]] / np.sqrt(2))}
    
    inverse_unitary = np.eye(2)
    for g in sequence:
        inverse_unitary = unitaries[g] @ inverse_unitary

    qc_unitary_inverse.unitary(inverse_unitary.conj().T,0)
#    print(type(qc_unitary_inverse))
    
    return qc_unitary_inverse


############
def inverse_map(X: np.array) -> np.array:
    if isUnitary(X):
        return X.conj().T
    return X.conj().T

def isUnitary(X: np.array) -> bool:
    return np.allclose(np.eye(len(X)), X @ X.conj().T)